# ****Question-1****

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, input_shape, num_classes, num_filters, filter_size, activation_conv, activation_dense, num_neurons_dense):
        super(CNN, self).__init__()
        self.conv_layers = self._create_conv_layers(input_shape[0], num_filters, filter_size, activation_conv)
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 7 * 7, num_neurons_dense),
            activation_dense,
            nn.Linear(num_neurons_dense, num_classes)
        )

    def _create_conv_layers(self, input_channels, num_filters, filter_size, activation_conv):
        layers = []
        in_channels = input_channels
        for _ in range(5):  # Reduced to 5 convolutional layers
            layers += [
                nn.Conv2d(in_channels, num_filters, filter_size, padding=1),
                activation_conv,
                nn.MaxPool2d(kernel_size=2, stride=2)
            ]
            in_channels = num_filters
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# Example parameters
input_shape = (3, 224, 224)  # Example shape compatible with iNaturalist dataset
num_classes = 10  # Number of classes in iNaturalist dataset
num_filters = 32  # Number of filters in convolutional layers
filter_size = 3  # Size of filters

# Define activation functions for convolutional and dense layers
activation_conv = nn.ReLU(inplace=True)  # Activation function for convolutional layers
activation_dense = nn.ReLU(inplace=True)  # Activation function for dense layer

num_neurons_dense = 1024  # Number of neurons in dense layer

# Create the model
model = CNN(input_shape, num_classes, num_filters, filter_size, activation_conv, activation_dense, num_neurons_dense)

# Display model summary
print(model)




CNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequenti

# **Question-2**

In [6]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import ConcatDataset
import torch.nn.functional as F
from PIL import Image
import os
import random
from collections import defaultdict  # Import defaultdict
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import Subset

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [55]:

# Define CNN architecture
class CNN(nn.Module):
    def __init__(self, input_channels, num_classes,num_filters,filter_organization,filter_size,activation,batch_normalization,dropout_value,num_nuerons):
        super(CNN, self).__init__()

        self.conv_layers = nn.ModuleList()  # ModuleList to store the convolutional layers

        # Define the convolutional layers dynamically using a loop
        padding=1
        
        in_channels = input_channels
        
        out_size = 224
        for i in range(5):
            self.conv_layers.append(nn.Conv2d(in_channels, num_filters, kernel_size=filter_size, padding=padding))
            if batch_normalization:
                self.conv_layers.append(nn.BatchNorm2d(num_filters))
            if activation == 'relu':
                self.conv_layers.append(nn.ReLU())  # Add ReLU activation
            elif activation == 'gelu':
                self.conv_layers.append(nn.GELU())  # Add GELU activation
            elif activation == 'silu':
                self.conv_layers.append(nn.SiLU())  # Add SiLU activation
            elif activation == 'mish':
                self.conv_layers.append(Mish())  # Add Mish activation
            self.conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            
            if(dropout_value > 0):
                self.conv_layers.append(nn.Dropout(dropout_value))
                
            
            in_channels = num_filters
            
            
            pool_kernel_size=2
            pool_stride=2
            
            # Update num_filters based on filter_organization
            if filter_organization == 'double':
                num_filters = int(num_filters * 2)
            elif filter_organization == 'halve':
                num_filters = int(num_filters / 2)
            
            
                    
            # Update out_size based on pooling parameters
            out_size = ((out_size + 2 * padding - pool_kernel_size) // pool_stride )+ 1
                
                
        
                
        
        

        self.fc1 = nn.Linear(in_channels * (out_size-1) * (out_size-1), num_nuerons)
        self.fc2 = nn.Linear(num_nuerons, num_classes)

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # Exclude batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))





In [47]:
def data_loader(train_data_folder,data_augmentation):
    
    
    without_augmentation_transform =  transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    # Load the original training dataset
    train_dataset = datasets.ImageFolder(root=train_data_folder, transform=without_augmentation_transform)

    # Shuffle the dataset
    indices = list(range(len(train_dataset)))
    np.random.shuffle(indices)
    
    # Calculate the size of the validation set (20% of the training data)
    val_size = int(0.2 * len(train_dataset))
    
    # Calculate the number of samples per class for validation
    num_classes = len(train_dataset.classes)
    val_size_per_class = val_size // num_classes
    
    # Initialize lists to store indices for training and validation
    train_indices = []
    val_indices = []
    
    # Iterate through each class to select validation samples
    for class_idx in range(num_classes):
        class_indices = [i for i in indices if train_dataset.targets[i] == class_idx]
        val_indices.extend(class_indices[:val_size_per_class])
        train_indices.extend(class_indices[val_size_per_class:])
    


    if data_augmentation:

        # Define data augmentation transforms for training data
        train_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize((224, 224)),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        # Create PyTorch data loaders for the initial dataset
        train_loader = DataLoader(Subset(train_dataset, train_indices), batch_size=64, shuffle=True)
        val_loader = DataLoader(Subset(train_dataset, val_indices), batch_size=64, shuffle=True)
        
        
        transformed_dataset = datasets.ImageFolder(root=train_data_folder, transform=train_transform)
        transformed_loader = DataLoader(Subset(transformed_dataset, train_indices), batch_size=64, shuffle=True)
        




  
        


        # Concatenate transformed datasets
        combined_train_dataset = ConcatDataset([train_loader.dataset,transformed_loader.dataset ])  # You can repeat train_dataset_transformed multiple times as needed
        
        # Define data loaders for combined datasets
        train_loader = DataLoader(dataset=combined_train_dataset, batch_size=64, shuffle=True)
    

    else:
        # Create PyTorch data loaders for the initial dataset
        train_loader = DataLoader(Subset(train_dataset, train_indices), batch_size=64, shuffle=True)
        val_loader = DataLoader(Subset(train_dataset, val_indices), batch_size=64, shuffle=True)


    
    return train_loader , val_loader

In [32]:
train_data_folder='/kaggle/input/inaturalist/inaturalist_12K/train'
data_augmentation=True
train_loader , val_loader=data_loader(train_data_folder,data_augmentation)

In [24]:
print(len(train_loader))
print(len(val_loader))
print(len(p))
print(len(q))
print(len(r))
print(len(s))
print(len(t))


251
32
9999
16018
8009
1990
9999


In [33]:

# Define training function
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total
    return train_loss, train_accuracy

# Define testing function
def validate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss = running_loss / len(val_loader)
    val_accuracy = correct / total
    return val_loss, val_accuracy


In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_data_folder='/kaggle/input/inaturalist/inaturalist_12K/train'

train_dataset = datasets.ImageFolder(root=train_data_folder)

train_loader , val_loader=data_loader(train_data_folder,data_augmentation)

# Model parameters
input_channels = 3
num_classes = len(train_dataset.classes)
num_filters = 32

# Choose activation function for convolutional layers
activation = 'mish'  # You can choose from 'ReLU', 'GELU', 'SiLU', 'Mish'

filter_organization='double' 


filter_size=3
batch_normalization=True
dropout_value=0.3
num_nuerons=128
data_augmentation=True


# Create model instance
model = CNN(input_channels, num_classes, num_filters,filter_organization,filter_size,activation,batch_normalization,dropout_value,num_nuerons).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)



# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = validate(model, val_loader , criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs},\n Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f},\n Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

In [61]:
print(val_accuracy)

0.22261306532663316


# Question 4

In [ ]:
# Define testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_loss = running_loss / len(test_loader)
    test_accuracy = correct / total
    return test_loss, test_accuracy

In [ ]:
without_augmentation_transform =  transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_data_folder='/kaggle/input/inaturalist/inaturalist_12K/val'

test_dataset = datasets.ImageFolder(root=test_data_folder,transform=without_augmentation_transform)
train_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

input_channels=3
num_classes=10
num_filters=32
filter_organization='double'
filter_size=3
activation='relu'
batch_normalization=True
dropout_value=0.3
num_nuerons=128

# Create model instance
best_model = CNN(input_channels, num_classes, num_filters,filter_organization,filter_size,activation,batch_normalization,dropout_value,num_nuerons).to(device)


test_loss, test_accuracy =test(best_model, test_loader, criterion, device)

print(f'Test Loss :{test_loss}')
print(f'Test Accuracy :{test_accuracy}')